In [19]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

In [22]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url

In [23]:
#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass


크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지:  500 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=SME&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=SME&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=SME&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=SME&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=SME&start=41', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=SME&start=51', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=SME&start=61', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=SME&start=71', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=SME&start=81', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=SME&start=91', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=SME&start=101', 'https://search.naver.com/search.naver?where=

100%|██████████| 257/257 [00:00<?, ?it/s]


In [25]:
# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")
    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

100%|██████████| 77/77 [00:18<00:00,  4.17it/s]

검색된 기사 갯수: 총  5000 개

[뉴스 제목]
['[단독] 중소기업 M&amp;A 활성화·스팩 통한 상장도 확대', '네이버로 ‘돈 버는 법’ 140만명이 무료로 수강…SME 상생 ‘박차’', 'KB금융그룹 새 회장에 양종희…비은행 강화 드라이브', 'SME 디지털 전환 보폭 넓히는 네이버… 온·오프 연계교육 넘어 산학협력·IP 컨설팅까지', '네이버, 10년간 140만 SME 및 창작자 디지털 전환 교육 도왔다', '\n\t\t\t신예 라미, ‘it’SMe’ 순수+청량\n\t\t', "네이버, SME 맞춤형 브랜딩 지원 프로그램 '브랜드 부스터' 시작", 'SME biz sentiment up for Sept. but still below par', "최수연 네이버 대표, 직원과 직접 소통으로 '프로젝트 꽃' 피워내", '\n\t\t\tSM 신예 라미, 동화 같은 비주얼…연기 활동 기대 UP\n\t\t', '윤곽 드러난 KB 차기 회장…‘허인 vs 양종희’ 2파전', 'Korean SMEs enjoy strong sales in Taiwan via Coupang', "네이버 직원 동기부여 비결…SME 상생 '프로젝트 꽃'", '네이버페이, 대안신용평가모형 케이뱅크와 협력...금융·결제서비스 외연 확장', '네이버 최수연 "AI 솔루션·브랜딩으로 SME 성장 돕겠다"', "[게시판] 네이버, 내달 10일까지 '브랜드 부스터' 참가자 모집", '韓 최대 플랫폼은 ‘변화 중’…‘위기’ 네이버가 내놓은 두 가지 해답 [기승전-플랫폼]', '네이버, 성장 단계별 맞춤형 브랜딩 지원 프로그램 강화', "[대전환 AI시대⑪] 하정우 네이버클라우드 센터장이 본 '초거대 AI' 실상", '창업부터 브랜딩까지 전폭 지원…네이버는 중소상공인 브랜드화에 ‘진심’', '최수연 네이버 대표 "중소상공인 지원 규모 확대에 집중"', '최수연 네이버 대표 "AI 솔루션과 브랜딩으로 SME 지속 가능한 성장 도모"', '[CEO열전]황현식 LGU+ 대표, 고객경험 혁신으로 성장 박

In [31]:
###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})
news_df

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

ImportError: cannot import name 'ArrowDtype' from 'pandas.core.dtypes.dtypes' (c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\dtypes.py)